In [2]:
pip install google-genai

  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.13.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (

In [ ]:
def get_md_files(directory):
    md_files = []
    # Get all .md files in current directory
    md_files.extend(glob.glob(os.path.join(directory, '*.md')))
    # Look for subdirectories
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            # Recursively get .md files from subdirectories
            md_files.extend(get_md_files(item_path))
    return md_files

In [ ]:
import base64
import os
from google import genai
from google.genai import types


def generate():
    prompt_content=""
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    with open('prompt.txt', 'r') as file:
        prompt_content = file.read()
    
    docs_path = 'docs/chat'
    markdown_files = get_md_files(docs_path)
    
    # Format the list of files for output
    file_list = []
    for file_path in markdown_files:
        relative_path = os.path.relpath(file_path, docs_path)
        file_list.append(relative_path)


    model = "gemini-2.5-pro-exp-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""How do I integrate LikeMinds Chat SDK in Flutter?"""),
            ],
        ),
    ]
    tools = [
        types.Tool(
            function_declarations=[
                types.FunctionDeclaration(
                    name="fetch_documents",
                    description="Fetches the documentation needed for the context, given the tech stack",
                    parameters=types.FunctionParameters(
                        properties={
                            "tech_stack": types.Property(
                                description="The tech stack of the project",
                                type="string",
                            ),
                        },
                    ),
                    response=types.FunctionResponse(
                        mime_type="text/plain",
                        parts=[
                            types.Part.from_text(text="""INSERT_RESPONSE_HERE"""),
                        ],
                    ),
                ),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        tools=tools,
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text="""You are a helpful integration assistant from LikeMinds, which is a company that makes Chat and Feed SDKs in multiple tech stacks (React, React Native, Flutter, Android, and iOS). You are an expert at preparing solutions and integration guides and runnable code in all our supported SDKs. You have access to our documentation which details how everything is supposed to be done. Do not hallucinate any information, provide clear and concise steps."""),
            types.Part.from_text(text=prompt_content)
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text if not chunk.function_calls else chunk.function_calls[0])

if __name__ == "__main__":
    generate()
